In [5]:
import pandas as pd
from collections import deque

В файле содержится информация о покупках людей.
id – означает покупку (в одну покупку входят все товары, купленные пользователем во время 1 похода в магазин)
Товар – наименование товара
Количество – число единиц купленного товара
Воспользуйтесь этими данными и выясните, какие пары товаров пользователи чаще всего покупают вместе. По сути, вам необходимо найти паттерны покупок, что позволит оптимизировать размещение продуктов в магазине, для удобства пользователей и увеличения выручки.


In [6]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/409319/test1_completed.csv')

 Изучаем данные

In [7]:
df.head()

,id,Товар,Количество
0,17119,Лимон,1.1
1,17119,Лимон оранжевый,0.7
2,17119,Лук-порей,10.0
3,17119,Лук репчатый,2.5
4,17119,Малина свежая,1.0


In [8]:
df.shape

(43514, 3)

In [9]:
df.isna().sum()

id            0
Товар         0
Количество    0
dtype: int64

In [10]:
df.nunique()

id            3273
Товар          199
Количество     101
dtype: int64

In [11]:
df.dtypes

id              int64
Товар          object
Количество    float64
dtype: object

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43514 entries, 0 to 43513
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          43514 non-null  int64  
 1   Товар       43514 non-null  object 
 2   Количество  43514 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1020.0+ KB


In [13]:
df.rename(columns={'Товар': 'article', 'Количество': 'quantity'},inplace=True)  

In [14]:
df.query('quantity == 0') 

,id,article,quantity


In [15]:
del df['quantity'] 

In [16]:
df_one_article = df.groupby(['id'], as_index=False).agg({'article': 'count'}).sort_values('article').query('article == 1') 

In [17]:
only_one_article = df_one_article.id.to_list()

In [18]:
new_df = df[~df.id.isin(only_one_article)] 

In [19]:
new_df = new_df.reset_index(drop=True)

In [20]:
new_df.groupby(['id'], as_index=False).agg({'article': 'count'}).sort_values('article') 

,id,article
671,39472,2
1677,71763,2
1669,71500,2
2576,100639,2
1636,70437,2
...,...,...
1828,77120,56
2041,83977,59
153,23467,61
648,38766,62


In [21]:
(df.shape[0]- new_df.shape[0] == len(only_one_article))

True

In [22]:
# заполняем словарь ключами id и значениями article
onecheck = {}
for i in range(len(new_df)):
    if new_df.id[i] not in onecheck:
        onecheck[new_df.id[i]] = []
    onecheck[new_df.id[i]].append(new_df.article[i])

In [24]:
# Создаем словарь, в котором будем хранить пары слов
pair_articles = {}
for i in (onecheck):
    pair_articles[i] = []

In [25]:
# заполняем словарь всеми вариантами пар слов из одной покупки
for i in (onecheck):
    onecheck[i] = sorted(onecheck[i])
    onecheck[i] = deque(onecheck[i])
    for j in range(len(onecheck[i])):
        for k in range(len(onecheck[i])):
            if k < len(onecheck[i]) - 1:
                pair_articles[i].append(onecheck[i][0] + ' ' + onecheck[i][k + 1])
        onecheck[i].popleft()

In [26]:
len(pair_articles) == len(onecheck)

True

In [27]:
# создаем новый словарь, ключем в котором являются пары, а значением их частота
pair_count = {}
for i in (pair_articles.values()):
    for j in i:
        if j not in pair_count:
            pair_count[j] = 0
        pair_count[j] += 1

In [28]:
df_pair = pd.DataFrame.from_dict(pair_count, orient='index')

In [29]:
df_pair.rename(columns={0: 'count'}, inplace=True)

In [30]:
df_pair.sort_values('count', ascending=False).head(5)  # выводим топ 5 самых вcnречаемых пар

,count
Огурцы Луховицкие Укроп,431
Петрушка Укроп,408
Арбуз Огурцы Луховицкие,345
Кабачки Огурцы Луховицкие,326
Кинза Укроп,303


Промежуточный итог.
На основании полученых данных можно сделать вывод о совместном раположении овощей и зелени. Хотелось бы сделать более полезные выводы для чего повнимательнее рассмотрим данные.

In [31]:
products = df.groupby('article', as_index=False)\
    .agg(sum).article.to_list()

In [32]:
products

['Абрикос вяленый',
 'Абрикосы молдавские',
 'Авокадо ХАСС',
 'Авокадо стандарт',
 'Алыча вяленая',
 'Ананас Gold',
 'Ананасовые кольца',
 'Апельсины столовые',
 'Арбуз',
 'Арбуз овальный',
 'Базилик зеленый',
 'Базилик красный',
 'Баклажаны белые',
 'Баклажаны грунтовые',
 'Банановые чипсы',
 'Бананы',
 'Бананы мини',
 'Барбарис сушеный',
 'Боярышник сушеный',
 'Бразильский орех',
 'Брусника свежая',
 'Букет "Брокколи"',
 'Букет "Закат"',
 'Букет "Кассиопея"',
 'Букет "Колорит"',
 'Букет "Оазис"',
 'Букет "Оранж"',
 'Букет "Фаворит"',
 'Вишня вяленая',
 'Вишня садовая',
 'Галангал корень',
 'Горох стручковый',
 'Гранат',
 'Грейпфрут красный',
 'Грибы Еринги',
 'Грибы вешенки',
 'Грибы лисички',
 'Грибы шампиньоны',
 'Груша Аббат',
 'Груша Конференц',
 'Груша Пакхам',
 'Груша Форел',
 'Груша вяленая',
 'Груша садовая',
 'Дыня "Косичка"',
 'Дыня Гуляби',
 'Дыня Колхозница',
 'Дыня цукаты',
 'Ежевика свежая',
 'Изюм Голд',
 'Изюм Малаяр',
 'Изюм Султана',
 'Имбирь корень',
 'Инжир свежий

In [33]:
df.query('article == "Подпись открытки"')  # некоторые позиции в данных стоит обсудить с колегами, предоставившими данные

,id,article
27,17618,Подпись открытки
344,19034,Подпись открытки
548,19559,Подпись открытки
674,20184,Подпись открытки
735,20289,Подпись открытки
...,...,...
42446,115851,Подпись открытки
42615,116407,Подпись открытки
42859,116974,Подпись открытки
43424,119062,Подпись открытки


В продуктовых магазинах как правило продукты находятся по группам товаров и для совместного раположения нам не так важны сорта помидор или яблок. Поэтому попробуем объединить самые частыо встречаемые слова и посмотреть что изменится

In [34]:
def group_word(word):
    if 'сушен' not in word:
        if word.startswith('Яблоки'):
            word = 'Яблоки'
        elif word.startswith('Салат'):
            word = 'Салат'
        elif word.startswith('Букет'):
            word = 'Букет'
        elif word.startswith('Груша'):
            word = 'Груша'
        elif word.startswith('Грибы'):
            word = 'Грибы'
        elif word.startswith('Огурцы'):
            word = 'Огурцы'
        elif word.startswith('Помидоры'):
            word = 'Помидоры'

    return word

In [35]:
new_df['group'] = new_df.article.apply(group_word)

In [36]:
len(new_df.group.unique())

160

In [37]:
new_df

,id,article,group
0,17119,Лимон,Лимон
1,17119,Лимон оранжевый,Лимон оранжевый
2,17119,Лук-порей,Лук-порей
3,17119,Лук репчатый,Лук репчатый
4,17119,Малина свежая,Малина свежая
...,...,...,...
43370,119396,Укроп,Укроп
43371,119396,Свекла,Свекла
43372,119396,Физалис,Физалис
43373,119396,Черешня сушеная,Черешня сушеная


In [38]:
onecheck = {}
for i in range(len(new_df)):
    if new_df.id[i] not in onecheck:
        onecheck[new_df.id[i]] = []
    onecheck[new_df.id[i]].append(new_df.group[i])

In [39]:
pair_articles = {}
for i in (onecheck):
    pair_articles[i] = []

In [40]:
for i in (onecheck):
    onecheck[i] = sorted(onecheck[i])
    onecheck[i] = deque(onecheck[i])
    for j in range(len(onecheck[i])):
        for k in range(len(onecheck[i])):
            if k < len(onecheck[i]) - 1:
                pair_articles[i].append(onecheck[i][0] + ' ' + onecheck[i][k + 1])
        onecheck[i].popleft()

In [41]:
pair_count = {}
for i in (pair_articles.values()):
    for j in i:
        if j not in pair_count:
            pair_count[j] = 1
        pair_count[j] += 1

In [42]:
df_pair = pd.DataFrame.from_dict(pair_count, orient='index')

In [43]:
df_pair.rename(columns={0: 'count'}, inplace=True)

In [44]:
df_pair.sort_values('count',ascending=False).head(5)  # выводим топ 5 самых втсречаемых пар

,count
Огурцы Салат,1674
Огурцы Яблоки,1265
Салат Яблоки,1114
Груша Огурцы,965
Огурцы Помидоры,942


Выводы.
К сожалению, новые результаты дают те же выводы что и старые.
Можно заметить большое количеество пар овощей, зелени и фруктов. Возможно, это связано с сезонностью и тем. С целью получения новых закономерностей, можно попробовать ввести ранги продуктов, но это выходит за рамки текущего проекта и сроков.